<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/BB_seq_bn_P_3_seq_bn_C_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This creates one adapter for pre-training and another for classifcation.  

* The pre-training adapter is not trained during the classification, only the classifcation adapter.






# Setup

In [ ]:
from time import time
start_time = time()

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

In [ ]:
!pip install pynvml

In [ ]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Pretraining

In [1]:
from datasets import load_dataset

pretrain_train_dataset, pretrain_eval_dataset = load_dataset("BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset", split=["test","dev"])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

/Users/afm/.pyenv/versions/3.11.3/envs/adapt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9754
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1980
})


In [2]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/Users/afm/.pyenv/versions/3.11.3/envs/adapt/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/afm/.pyenv/versions/3.11.3/envs/adapt/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/afm/.pyenv/versions/3.11.3/envs/adapt/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.poo

In [3]:
# THIS MUST BE SET TO MATCH  pretraining_adapter_type
from adapters import SeqBnConfig

pretraining_adapter_config = SeqBnConfig()
pretrained_adapter_name = "tapt_seq_bn"
pretrained_adapter_hub_name = f"pretrain_{pretrained_adapter_name}_adpater"

In [4]:
print(pretraining_adapter_config.__class__.__name__)
print(pretrained_adapter_name)

SeqBnConfig
tapt_seq_bn


In [5]:


# Add a new adapter
pretrain_model.add_adapter(pretrained_adapter_name, config=pretraining_adapter_config)

# Add head for masked language modeling
pretrain_model.add_masked_lm_head(pretrained_adapter_name)

# Set the adapter to be used for training
pretrain_model.train_adapter(pretrained_adapter_name)

print(f"Pretrained Adapter Name: {pretrained_adapter_name}")

Pretrained Adapter Name: tapt_seq_bn


In [6]:
pretrain_model.active_adapters

Stack[tapt_seq_bn]

In [7]:
summary = pretrain_model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
tapt_seq_bn              bottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [9]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [10]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir="tapt_helpfulness_pretraining_model",
    learning_rate=0.0005, # Paper for DAPT training
    per_device_train_batch_size=32, # 346,m - TRYING SMALLER BATCH SIZE
    per_device_eval_batch_size=32, #346,m - TRYING SMALLER BATCH SIZE
    gradient_accumulation_steps=11, # Creates effective batch of 32*11*6 = 2112
    num_train_epochs=2, # Just one epoch for one pass on data
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="steps",
    eval_steps=500, # Evaluates every 50 steps
    save_strategy="steps",
    save_steps=500, # Checkpoint saves every 500 steps / every 1,000,000 reviews ( 500 * 10 * 34 * 6)
    save_total_limit=2, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progress
)

In [11]:
from adapters import AdapterTrainer

pretrain_trainer = AdapterTrainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
)

/Users/afm/.pyenv/versions/3.11.3/envs/adapt/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [12]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

  0%|          | 0/54 [00:00<?, ?it/s]You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: MPS backend out of memory (MPS allocated: 32.16 GB, other allocations: 3.49 GB, max allowed: 36.27 GB). Tried to allocate 3.07 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
after_pretrain_eval_results = pretrain_trainer.evaluate()
after_pretrain_eval_results_loss = after_pretrain_eval_results["eval_loss"]
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print(after_pretrain_eval_results)

In [ ]:
# pretrain_model.push_adapter_to_hub(
#     pretrained_adapter_hub_name,
#     pretrained_adapter_name,
#     datasets_tag=pretrain_dataset_name
# )

# Classification

In [ ]:
# print("Disconnecting Session")
# from google.colab import runtime
# runtime.unassign()